In [ ]:
# 🎓 K-12 Homework Analytics with Snowflake Cortex AISQL

This notebook demonstrates the powerful new **Cortex AISQL functions** using a realistic K-12 homework management scenario.

## 📋 What We'll Cover
- **AI_AGG**: Aggregate homework reviews and feedback
- **AI_CLASSIFY**: Classify homework sentiment and categories
- **AI_COMPLETE**: Generate teacher insights and recommendations
- **Complex Joins**: Multi-table analysis across schools, students, and assignments
- **Visualizations**: Grade distributions and sentiment analysis

## 🏫 Our Scenario
We're analyzing data from **Lincoln Elementary School** with:
- Multiple grade levels (K-5)
- 5 core subjects (Math, Science, English, Social Studies, Art)
- ~30 students per class
- Homework submissions, grades, and feedback


In [ ]:
## 🔧 Environment Setup

First, let's set up our database and import required libraries.


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import streamlit as st
import plotly.express as px
import plotly.graph_objects as go
from faker import Faker

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)
fake = Faker()
Faker.seed(42)

print("Libraries imported successfully! 🚀")


In [ ]:
## 🗄️ Database Setup

Create our demo database and schema structure.


In [ ]:
-- Create demo database and schema
CREATE OR REPLACE DATABASE K12_HOMEWORK_DEMO;
USE DATABASE K12_HOMEWORK_DEMO;
CREATE OR REPLACE SCHEMA EDUCATION_DATA;
USE SCHEMA EDUCATION_DATA;

-- Confirmation message
SELECT 'Database and schema created successfully! 🎉' AS status;


In [ ]:
## 📊 Table Creation

Let's create our core tables for the K-12 homework system.


In [ ]:
-- Create core tables for K-12 homework system

-- Schools table
CREATE OR REPLACE TABLE schools (
    school_id INT PRIMARY KEY,
    school_name VARCHAR(100),
    district VARCHAR(100),
    state VARCHAR(2),
    enrollment_count INT
);

-- Grade levels table
CREATE OR REPLACE TABLE grade_levels (
    grade_id INT PRIMARY KEY,
    grade_name VARCHAR(20),
    grade_level INT
);

-- Subjects table
CREATE OR REPLACE TABLE subjects (
    subject_id INT PRIMARY KEY,
    subject_name VARCHAR(50),
    department VARCHAR(50)
);

-- Students table
CREATE OR REPLACE TABLE students (
    student_id INT PRIMARY KEY,
    school_id INT,
    grade_id INT,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    enrollment_date DATE
);

-- Classrooms table
CREATE OR REPLACE TABLE classrooms (
    classroom_id INT PRIMARY KEY,
    school_id INT,
    grade_id INT,
    subject_id INT,
    teacher_name VARCHAR(100),
    classroom_size INT
);

-- Homework assignments table
CREATE OR REPLACE TABLE homework_assignments (
    assignment_id INT PRIMARY KEY,
    classroom_id INT,
    assignment_name VARCHAR(200),
    assignment_type VARCHAR(50),
    due_date DATE,
    max_points INT,
    instructions TEXT
);

-- Homework submissions table
CREATE OR REPLACE TABLE homework_submissions (
    submission_id INT PRIMARY KEY,
    assignment_id INT,
    student_id INT,
    submitted_date TIMESTAMP,
    grade_received FLOAT,
    submission_status VARCHAR(20)
);

-- Homework feedback table
CREATE OR REPLACE TABLE homework_feedback (
    feedback_id INT PRIMARY KEY,
    submission_id INT,
    teacher_feedback TEXT,
    student_reflection TEXT,
    parent_comment TEXT,
    feedback_date TIMESTAMP
);

SELECT 'All tables created successfully! 📚' AS status;


In [ ]:
## 🎲 Data Generation

Now let's populate our tables with realistic K-12 homework data.


In [ ]:
-- Insert reference data

-- Insert school data
INSERT INTO schools VALUES (1, 'Lincoln Elementary School', 'Springfield District', 'IL', 450);

-- Insert grade levels
INSERT INTO grade_levels VALUES 
    (1, 'Kindergarten', 0),
    (2, '1st Grade', 1),
    (3, '2nd Grade', 2),
    (4, '3rd Grade', 3),
    (5, '4th Grade', 4),
    (6, '5th Grade', 5);

-- Insert subjects
INSERT INTO subjects VALUES 
    (1, 'Mathematics', 'STEM'),
    (2, 'Science', 'STEM'),
    (3, 'English Language Arts', 'Language Arts'),
    (4, 'Social Studies', 'Social Sciences'),
    (5, 'Art', 'Fine Arts');

-- Verify data insertion
SELECT 'Reference data inserted successfully! 📝' AS status;
SELECT * FROM schools;
SELECT * FROM grade_levels;
SELECT * FROM subjects;


In [ ]:
# Generate realistic classroom and student data

# Teacher names for assignments
teacher_names = [
    'Ms. Johnson', 'Mr. Smith', 'Mrs. Williams', 'Ms. Brown', 'Mr. Davis',
    'Mrs. Miller', 'Ms. Wilson', 'Mr. Moore', 'Mrs. Taylor', 'Ms. Anderson',
    'Mr. Thomas', 'Mrs. Jackson', 'Ms. White', 'Mr. Harris', 'Mrs. Martin',
    'Ms. Thompson', 'Mr. Garcia', 'Mrs. Martinez', 'Ms. Robinson', 'Mr. Clark',
    'Mrs. Rodriguez', 'Ms. Lewis', 'Mr. Lee', 'Mrs. Walker', 'Ms. Hall',
    'Mr. Allen', 'Mrs. Young', 'Ms. Hernandez', 'Mr. King', 'Mrs. Wright'
]

# Generate sample homework feedback
homework_feedback_samples = [
    "Great work on this assignment! Your understanding of the concepts is evident.",
    "Good effort, but please review the instructions more carefully next time.",
    "Excellent problem-solving approach. Keep up the good work!",
    "This assignment shows improvement from your previous work.",
    "Please see me after class to discuss some areas for improvement.",
    "Outstanding creativity and attention to detail!",
    "Your work demonstrates solid understanding of the material.",
    "Good job, but remember to show your work on math problems.",
    "This assignment reflects careful thought and preparation.",
    "Well done! Your explanations are clear and thorough."
]

# Generate sample student reflections
student_reflections = [
    "I found this assignment challenging but learned a lot.",
    "I enjoyed working on this project and feel confident about the topic.",
    "I struggled with some parts but asked for help when needed.",
    "This assignment helped me understand the material better.",
    "I spent extra time on this and feel proud of my work.",
    "I need to practice more on this topic.",
    "Working on this assignment was fun and educational.",
    "I found the instructions clear and easy to follow.",
    "This assignment was difficult but I didn't give up.",
    "I learned new strategies while working on this."
]

# Generate sample parent comments
parent_comments = [
    "Thank you for the detailed feedback on my child's work.",
    "We appreciate the extra support you provide in class.",
    "My child really enjoys your subject and talks about it at home.",
    "Please let us know if there are areas where we can help at home.",
    "Thank you for encouraging my child to do their best.",
    "The homework assignments are appropriately challenging.",
    "We're happy to see our child's progress in this subject.",
    "Your teaching methods really connect with my child.",
    "Thank you for being patient and supportive.",
    "The feedback helps us understand how to support learning at home."
]

print("Sample data generated successfully! 🎨")
print(f"Generated {len(teacher_names)} teacher names")
print(f"Generated {len(homework_feedback_samples)} feedback samples")
print(f"Generated {len(student_reflections)} student reflection samples")
print(f"Generated {len(parent_comments)} parent comment samples")


In [ ]:
-- Insert sample classrooms (30 total: 6 grades × 5 subjects)
INSERT INTO classrooms VALUES
(1, 1, 1, 1, 'Ms. Johnson', 28), (2, 1, 1, 2, 'Mr. Smith', 30), (3, 1, 1, 3, 'Mrs. Williams', 29), (4, 1, 1, 4, 'Ms. Brown', 31), (5, 1, 1, 5, 'Mr. Davis', 27),
(6, 1, 2, 1, 'Mrs. Miller', 32), (7, 1, 2, 2, 'Ms. Wilson', 28), (8, 1, 2, 3, 'Mr. Moore', 30), (9, 1, 2, 4, 'Mrs. Taylor', 29), (10, 1, 2, 5, 'Ms. Anderson', 31),
(11, 1, 3, 1, 'Mr. Thomas', 33), (12, 1, 3, 2, 'Mrs. Jackson', 29), (13, 1, 3, 3, 'Ms. White', 30), (14, 1, 3, 4, 'Mr. Harris', 28), (15, 1, 3, 5, 'Mrs. Martin', 32),
(16, 1, 4, 1, 'Ms. Thompson', 31), (17, 1, 4, 2, 'Mr. Garcia', 29), (18, 1, 4, 3, 'Mrs. Martinez', 30), (19, 1, 4, 4, 'Ms. Robinson', 28), (20, 1, 4, 5, 'Mr. Clark', 33),
(21, 1, 5, 1, 'Mrs. Rodriguez', 30), (22, 1, 5, 2, 'Ms. Lewis', 29), (23, 1, 5, 3, 'Mr. Lee', 31), (24, 1, 5, 4, 'Mrs. Walker', 28), (25, 1, 5, 5, 'Ms. Hall', 32),
(26, 1, 6, 1, 'Mr. Allen', 30), (27, 1, 6, 2, 'Mrs. Young', 29), (28, 1, 6, 3, 'Ms. Hernandez', 31), (29, 1, 6, 4, 'Mr. King', 28), (30, 1, 6, 5, 'Mrs. Wright', 33);

-- Insert sample students (30 per grade level)
INSERT INTO students VALUES
-- Kindergarten students
(1, 1, 1, 'Emma', 'Smith', '2023-08-15'), (2, 1, 1, 'Liam', 'Johnson', '2023-08-16'), (3, 1, 1, 'Olivia', 'Williams', '2023-08-17'), (4, 1, 1, 'Noah', 'Brown', '2023-08-18'), (5, 1, 1, 'Ava', 'Jones', '2023-08-19'),
(6, 1, 1, 'Ethan', 'Garcia', '2023-08-20'), (7, 1, 1, 'Sophia', 'Miller', '2023-08-21'), (8, 1, 1, 'Mason', 'Davis', '2023-08-22'), (9, 1, 1, 'Isabella', 'Rodriguez', '2023-08-23'), (10, 1, 1, 'William', 'Martinez', '2023-08-24'),
(11, 1, 1, 'Mia', 'Hernandez', '2023-08-25'), (12, 1, 1, 'James', 'Lopez', '2023-08-26'), (13, 1, 1, 'Charlotte', 'Gonzalez', '2023-08-27'), (14, 1, 1, 'Benjamin', 'Wilson', '2023-08-28'), (15, 1, 1, 'Amelia', 'Anderson', '2023-08-29'),
(16, 1, 1, 'Jacob', 'Thomas', '2023-08-30'), (17, 1, 1, 'Harper', 'Taylor', '2023-08-31'), (18, 1, 1, 'Michael', 'Moore', '2023-09-01'), (19, 1, 1, 'Evelyn', 'Jackson', '2023-09-02'), (20, 1, 1, 'Elijah', 'Martin', '2023-09-03'),
(21, 1, 1, 'Abigail', 'Lee', '2023-09-04'), (22, 1, 1, 'Owen', 'Perez', '2023-09-05'), (23, 1, 1, 'Emily', 'Thompson', '2023-09-06'), (24, 1, 1, 'Alexander', 'White', '2023-09-07'), (25, 1, 1, 'Elizabeth', 'Harris', '2023-09-08'),
(26, 1, 1, 'Daniel', 'Sanchez', '2023-09-09'), (27, 1, 1, 'Mila', 'Clark', '2023-09-10'), (28, 1, 1, 'Lucas', 'Ramirez', '2023-09-11'), (29, 1, 1, 'Ella', 'Lewis', '2023-09-12'), (30, 1, 1, 'Matthew', 'Robinson', '2023-09-13');

-- Insert students for other grades (abbreviated for demo)
INSERT INTO students VALUES
-- 1st Grade
(31, 1, 2, 'Grace', 'Walker', '2023-08-15'), (32, 1, 2, 'Henry', 'Young', '2023-08-16'), (33, 1, 2, 'Zoey', 'Allen', '2023-08-17'), (34, 1, 2, 'Jack', 'King', '2023-08-18'), (35, 1, 2, 'Lily', 'Wright', '2023-08-19'),
(36, 1, 2, 'Ryan', 'Scott', '2023-08-20'), (37, 1, 2, 'Chloe', 'Green', '2023-08-21'), (38, 1, 2, 'Connor', 'Adams', '2023-08-22'), (39, 1, 2, 'Aria', 'Baker', '2023-08-23'), (40, 1, 2, 'Hunter', 'Hill', '2023-08-24'),
(41, 1, 2, 'Layla', 'Nelson', '2023-08-25'), (42, 1, 2, 'Wyatt', 'Carter', '2023-08-26'), (43, 1, 2, 'Zoe', 'Mitchell', '2023-08-27'), (44, 1, 2, 'Caleb', 'Roberts', '2023-08-28'), (45, 1, 2, 'Nora', 'Turner', '2023-08-29'),
(46, 1, 2, 'Luke', 'Phillips', '2023-08-30'), (47, 1, 2, 'Stella', 'Campbell', '2023-08-31'), (48, 1, 2, 'Nathan', 'Parker', '2023-09-01'), (49, 1, 2, 'Hazel', 'Evans', '2023-09-02'), (50, 1, 2, 'Isaac', 'Edwards', '2023-09-03'),
(51, 1, 2, 'Violet', 'Collins', '2023-09-04'), (52, 1, 2, 'Grayson', 'Stewart', '2023-09-05'), (53, 1, 2, 'Aurora', 'Sanchez', '2023-09-06'), (54, 1, 2, 'Leo', 'Morris', '2023-09-07'), (55, 1, 2, 'Savannah', 'Rogers', '2023-09-08'),
(56, 1, 2, 'Ezra', 'Reed', '2023-09-09'), (57, 1, 2, 'Brooklyn', 'Cook', '2023-09-10'), (58, 1, 2, 'Mateo', 'Morgan', '2023-09-11'), (59, 1, 2, 'Bella', 'Bell', '2023-09-12'), (60, 1, 2, 'Aiden', 'Murphy', '2023-09-13');

-- Continue with grades 2-5 (abbreviated)
INSERT INTO students VALUES
-- 2nd Grade
(61, 1, 3, 'Luna', 'Bailey', '2023-08-15'), (62, 1, 3, 'Jayden', 'Rivera', '2023-08-16'), (63, 1, 3, 'Penelope', 'Cooper', '2023-08-17'), (64, 1, 3, 'Sebastian', 'Richardson', '2023-08-18'), (65, 1, 3, 'Leah', 'Cox', '2023-08-19'),
(66, 1, 3, 'Jaxon', 'Howard', '2023-08-20'), (67, 1, 3, 'Addison', 'Ward', '2023-08-21'), (68, 1, 3, 'Brayden', 'Torres', '2023-08-22'), (69, 1, 3, 'Aubrey', 'Peterson', '2023-08-23'), (70, 1, 3, 'Ian', 'Gray', '2023-08-24'),
(71, 1, 3, 'Skylar', 'Ramirez', '2023-08-25'), (72, 1, 3, 'Carson', 'James', '2023-08-26'), (73, 1, 3, 'Paisley', 'Watson', '2023-08-27'), (74, 1, 3, 'Jordan', 'Brooks', '2023-08-28'), (75, 1, 3, 'Naomi', 'Kelly', '2023-08-29'),
(76, 1, 3, 'Easton', 'Sanders', '2023-08-30'), (77, 1, 3, 'Peyton', 'Price', '2023-08-31'), (78, 1, 3, 'Kai', 'Bennett', '2023-09-01'), (79, 1, 3, 'Kinsley', 'Wood', '2023-09-02'), (80, 1, 3, 'Colton', 'Barnes', '2023-09-03'),
(81, 1, 3, 'Willow', 'Ross', '2023-09-04'), (82, 1, 3, 'Levi', 'Henderson', '2023-09-05'), (83, 1, 3, 'Ruby', 'Coleman', '2023-09-06'), (84, 1, 3, 'Maverick', 'Jenkins', '2023-09-07'), (85, 1, 3, 'Piper', 'Perry', '2023-09-08'),
(86, 1, 3, 'Asher', 'Powell', '2023-09-09'), (87, 1, 3, 'Ivy', 'Long', '2023-09-10'), (88, 1, 3, 'Jace', 'Patterson', '2023-09-11'), (89, 1, 3, 'Genesis', 'Hughes', '2023-09-12'), (90, 1, 3, 'Ryder', 'Flores', '2023-09-13');

SELECT 'Sample data inserted successfully! 🎒' AS status;
SELECT COUNT(*) as classroom_count FROM classrooms;
SELECT COUNT(*) as student_count FROM students;


In [ ]:
-- Insert homework assignments and submissions with feedback
INSERT INTO homework_assignments VALUES
-- Math assignments
(1, 1, 'Basic Addition Practice', 'Worksheet', '2024-01-15', 20, 'Complete addition problems 1-20'),
(2, 6, 'Multiplication Tables', 'Homework', '2024-01-20', 25, 'Practice multiplication tables 1-12'),
(3, 11, 'Fraction Worksheets', 'Worksheet', '2024-01-18', 30, 'Solve fraction problems and show work'),
-- Science assignments
(4, 2, 'Plant Growth Observation', 'Project', '2024-01-25', 40, 'Observe and record plant growth daily'),
(5, 7, 'Weather Tracking', 'Project', '2024-01-22', 35, 'Track weather patterns for one week'),
(6, 12, 'Simple Machines', 'Homework', '2024-01-19', 25, 'Identify simple machines in everyday objects'),
-- English assignments
(7, 3, 'Reading Comprehension', 'Worksheet', '2024-01-16', 30, 'Read passage and answer questions'),
(8, 8, 'Creative Writing', 'Project', '2024-01-24', 45, 'Write a short story about your favorite animal'),
(9, 13, 'Grammar Practice', 'Homework', '2024-01-21', 20, 'Complete grammar exercises in workbook'),
-- Social Studies assignments
(10, 4, 'Community Helpers', 'Project', '2024-01-26', 35, 'Research and present on a community helper'),
(11, 9, 'State History', 'Homework', '2024-01-23', 30, 'Learn about Illinois state history'),
(12, 14, 'Map Skills', 'Worksheet', '2024-01-17', 25, 'Practice reading maps and using compass directions'),
-- Art assignments
(13, 5, 'Color Theory', 'Project', '2024-01-27', 40, 'Create artwork demonstrating primary and secondary colors'),
(14, 10, 'Drawing Techniques', 'Homework', '2024-01-20', 30, 'Practice different drawing techniques'),
(15, 15, 'Sculpture Project', 'Project', '2024-01-28', 50, 'Create a sculpture using recyclable materials');

-- Insert sample homework submissions
INSERT INTO homework_submissions VALUES
-- Math submissions
(1, 1, 1, '2024-01-15 10:30:00', 85.0, 'Completed'),
(2, 1, 2, '2024-01-15 11:15:00', 92.5, 'Completed'),
(3, 1, 3, '2024-01-15 09:45:00', 78.0, 'Completed'),
(4, 2, 31, '2024-01-20 14:20:00', 88.0, 'Completed'),
(5, 2, 32, '2024-01-20 15:10:00', 95.0, 'Completed'),
(6, 3, 61, '2024-01-18 13:30:00', 82.5, 'Completed'),
-- Science submissions
(7, 4, 1, '2024-01-25 16:45:00', 90.0, 'Completed'),
(8, 4, 2, '2024-01-25 17:20:00', 87.5, 'Completed'),
(9, 5, 31, '2024-01-22 12:30:00', 93.0, 'Completed'),
(10, 6, 61, '2024-01-19 11:45:00', 85.5, 'Completed'),
-- English submissions
(11, 7, 1, '2024-01-16 08:30:00', 88.5, 'Completed'),
(12, 8, 31, '2024-01-24 19:15:00', 96.0, 'Completed'),
(13, 9, 61, '2024-01-21 14:45:00', 91.5, 'Completed'),
-- Social Studies submissions
(14, 10, 1, '2024-01-26 16:00:00', 89.0, 'Completed'),
(15, 11, 31, '2024-01-23 13:20:00', 84.5, 'Completed'),
-- Art submissions
(16, 13, 1, '2024-01-27 15:30:00', 94.0, 'Completed'),
(17, 14, 31, '2024-01-20 18:45:00', 87.0, 'Completed'),
(18, 15, 61, '2024-01-28 10:15:00', 92.5, 'Completed');

-- Insert sample feedback
INSERT INTO homework_feedback VALUES
(1, 1, 'Great work on this assignment! Your understanding of addition is evident.', 'I found this assignment easy and enjoyed doing the math problems.', 'Thank you for the clear instructions and positive feedback.', '2024-01-16 09:00:00'),
(2, 2, 'Excellent problem-solving approach. Keep up the good work!', 'I enjoyed working on this project and feel confident about addition.', 'We appreciate the detailed feedback on my child''s work.', '2024-01-16 09:30:00'),
(3, 3, 'Good effort, but please review the instructions more carefully next time.', 'I struggled with some parts but asked for help when needed.', 'Please let us know if there are areas where we can help at home.', '2024-01-16 10:00:00'),
(4, 4, 'Outstanding work on your multiplication tables!', 'I spent extra time practicing and feel proud of my work.', 'The homework assignments are appropriately challenging.', '2024-01-21 08:30:00'),
(5, 7, 'Your plant observation skills are impressive. Well documented!', 'This assignment helped me understand how plants grow.', 'My child really enjoys science and talks about it at home.', '2024-01-26 14:00:00'),
(6, 11, 'Creative and well-written story! You have a great imagination.', 'Working on this assignment was fun and I learned new words.', 'Thank you for encouraging creativity in your assignments.', '2024-01-17 11:30:00'),
(7, 12, 'Excellent creative writing! Your story was engaging and well-structured.', 'I enjoyed writing about my favorite animal and creating the story.', 'Your teaching methods really connect with my child.', '2024-01-25 16:00:00'),
(8, 14, 'Good presentation on community helpers. Clear and informative.', 'I learned a lot about how firefighters help our community.', 'We''re happy to see our child''s progress in social studies.', '2024-01-27 10:45:00'),
(9, 16, 'Beautiful artwork! Your understanding of color theory is excellent.', 'I had fun mixing colors and creating my artwork.', 'Thank you for fostering artistic expression in your class.', '2024-01-28 13:20:00'),
(10, 18, 'Creative sculpture project! Great use of recyclable materials.', 'This project was challenging but I didn''t give up and made something cool.', 'The feedback helps us understand how to support learning at home.', '2024-01-29 15:45:00');

SELECT 'All homework data inserted successfully! 📝' AS status;
SELECT COUNT(*) as assignment_count FROM homework_assignments;
SELECT COUNT(*) as submission_count FROM homework_submissions;
SELECT COUNT(*) as feedback_count FROM homework_feedback;


In [ ]:
## 🎯 Cortex AISQL Demonstrations

Now let's demonstrate the power of Snowflake's Cortex AISQL functions!


In [ ]:
### 📊 Before AISQL: Traditional Data Analysis

Let's first look at our raw data to understand what we're working with.


In [ ]:
-- Let's see our homework feedback data (before AI processing)
SELECT 
    hf.feedback_id,
    s.first_name || ' ' || s.last_name as student_name,
    sub.subject_name,
    gl.grade_name,
    ha.assignment_name,
    hs.grade_received,
    hf.teacher_feedback,
    hf.student_reflection,
    hf.parent_comment
FROM homework_feedback hf
JOIN homework_submissions hs ON hf.submission_id = hs.submission_id
JOIN homework_assignments ha ON hs.assignment_id = ha.assignment_id
JOIN classrooms c ON ha.classroom_id = c.classroom_id
JOIN students s ON hs.student_id = s.student_id
JOIN subjects sub ON c.subject_id = sub.subject_id
JOIN grade_levels gl ON s.grade_id = gl.grade_id
ORDER BY hf.feedback_id
LIMIT 10;


In [ ]:
### 🤖 AI_AGG Function: Intelligent Data Aggregation

**AI_AGG** allows us to aggregate text data using natural language instructions, perfect for summarizing feedback and reviews.


In [ ]:
-- Using AI_AGG to summarize teacher feedback by subject
SELECT 
    sub.subject_name,
    COUNT(*) as total_feedback,
    AI_AGG(hf.teacher_feedback, 'Summarize the main themes and concerns from teacher feedback') as feedback_summary,
    AI_AGG(hf.student_reflection, 'What are students saying about their learning experience?') as student_insights
FROM homework_feedback hf
JOIN homework_submissions hs ON hf.submission_id = hs.submission_id
JOIN homework_assignments ha ON hs.assignment_id = ha.assignment_id
JOIN classrooms c ON ha.classroom_id = c.classroom_id
JOIN subjects sub ON c.subject_id = sub.subject_id
WHERE hf.teacher_feedback IS NOT NULL
GROUP BY sub.subject_name
ORDER BY total_feedback DESC;


In [ ]:
-- Using AI_AGG to analyze parent engagement by grade level
SELECT 
    gl.grade_name,
    COUNT(*) as parent_comments_count,
    AI_AGG(hf.parent_comment, 'What are the main concerns and praises from parents?') as parent_sentiment_summary,
    ROUND(AVG(hs.grade_received), 2) as avg_grade
FROM homework_feedback hf
JOIN homework_submissions hs ON hf.submission_id = hs.submission_id
JOIN students s ON hs.student_id = s.student_id
JOIN grade_levels gl ON s.grade_id = gl.grade_id
WHERE hf.parent_comment IS NOT NULL
GROUP BY gl.grade_name, gl.grade_level
ORDER BY gl.grade_level;


In [ ]:
### 🏷️ AI_CLASSIFY Function: Intelligent Categorization

**AI_CLASSIFY** helps us categorize text data into meaningful groups, perfect for sentiment analysis and feedback classification.


In [ ]:
-- Using AI_CLASSIFY to categorize homework feedback sentiment
SELECT 
    hf.feedback_id,
    sub.subject_name,
    gl.grade_name,
    hs.grade_received,
    hf.teacher_feedback,
    AI_CLASSIFY(hf.teacher_feedback, ['Positive', 'Constructive', 'Needs Improvement', 'Excellent']) as feedback_sentiment,
    AI_CLASSIFY(hf.student_reflection, ['Confident', 'Struggling', 'Motivated', 'Frustrated']) as student_sentiment
FROM homework_feedback hf
JOIN homework_submissions hs ON hf.submission_id = hs.submission_id
JOIN homework_assignments ha ON hs.assignment_id = ha.assignment_id
JOIN classrooms c ON ha.classroom_id = c.classroom_id
JOIN subjects sub ON c.subject_id = sub.subject_id
JOIN students s ON hs.student_id = s.student_id
JOIN grade_levels gl ON s.grade_id = gl.grade_id
WHERE hf.teacher_feedback IS NOT NULL
ORDER BY hf.feedback_id
LIMIT 10;


In [ ]:
-- Sentiment analysis summary by subject
WITH classified_feedback AS (
    SELECT 
        sub.subject_name,
        AI_CLASSIFY(hf.teacher_feedback, ['Positive', 'Constructive', 'Needs Improvement', 'Excellent']) as feedback_sentiment,
        AI_CLASSIFY(hf.parent_comment, ['Supportive', 'Concerned', 'Satisfied', 'Frustrated']) as parent_sentiment
    FROM homework_feedback hf
    JOIN homework_submissions hs ON hf.submission_id = hs.submission_id
    JOIN homework_assignments ha ON hs.assignment_id = ha.assignment_id
    JOIN classrooms c ON ha.classroom_id = c.classroom_id
    JOIN subjects sub ON c.subject_id = sub.subject_id
    WHERE hf.teacher_feedback IS NOT NULL
)
SELECT 
    subject_name,
    feedback_sentiment,
    COUNT(*) as sentiment_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY subject_name), 2) as sentiment_percentage
FROM classified_feedback
GROUP BY subject_name, feedback_sentiment
ORDER BY subject_name, sentiment_count DESC;


In [ ]:
### 💡 AI_COMPLETE Function: Intelligent Content Generation

**AI_COMPLETE** generates contextual content based on our data, perfect for creating personalized recommendations and insights.


In [ ]:
-- Using AI_COMPLETE to generate personalized teacher recommendations
WITH student_performance AS (
    SELECT 
        s.student_id,
        s.first_name || ' ' || s.last_name as student_name,
        sub.subject_name,
        gl.grade_name,
        ROUND(AVG(hs.grade_received), 2) as avg_grade,
        hf.teacher_feedback
    FROM students s
    JOIN homework_submissions hs ON s.student_id = hs.student_id
    JOIN homework_assignments ha ON hs.assignment_id = ha.assignment_id
    JOIN classrooms c ON ha.classroom_id = c.classroom_id
    JOIN subjects sub ON c.subject_id = sub.subject_id
    JOIN grade_levels gl ON s.grade_id = gl.grade_id
    LEFT JOIN homework_feedback hf ON hs.submission_id = hf.submission_id
    WHERE hs.grade_received IS NOT NULL
    GROUP BY s.student_id, s.first_name, s.last_name, sub.subject_name, gl.grade_name, hf.teacher_feedback
    HAVING AVG(hs.grade_received) < 90  -- Focus on students who could improve
)
SELECT 
    student_name,
    subject_name,
    grade_name,
    avg_grade,
    AI_COMPLETE(
        'llama3.1-8b',
        'Based on student performance data: Subject: ' || subject_name || 
        ', Grade Level: ' || grade_name || 
        ', Average Grade: ' || avg_grade || 
        ', Recent Feedback: ' || COALESCE(teacher_feedback, 'No feedback') || 
        '. Generate 3 specific, actionable recommendations for improving this student''s performance.'
    ) as personalized_recommendations
FROM student_performance
LIMIT 5;


In [ ]:
-- Generate curriculum insights by subject
WITH subject_performance AS (
    SELECT 
        sub.subject_name,
        COUNT(DISTINCT s.student_id) as total_students,
        ROUND(AVG(hs.grade_received), 2) as avg_performance
    FROM subjects sub
    JOIN classrooms c ON sub.subject_id = c.subject_id
    JOIN homework_assignments ha ON c.classroom_id = ha.classroom_id
    JOIN homework_submissions hs ON ha.assignment_id = hs.assignment_id
    JOIN students s ON hs.student_id = s.student_id
    WHERE hs.grade_received IS NOT NULL
    GROUP BY sub.subject_id, sub.subject_name
)
SELECT 
    subject_name,
    total_students,
    avg_performance,
    AI_COMPLETE(
        'llama3.1-8b',
        'Analyze this K-12 subject performance data: Subject: ' || subject_name || 
        ', Total Students: ' || total_students || 
        ', Average Performance: ' || avg_performance || 
        '%. Generate insights about curriculum effectiveness and suggest 3 improvements for this subject area.'
    ) as curriculum_insights
FROM subject_performance
ORDER BY avg_performance DESC;


In [ ]:
## 🔗 Complex Multi-Table Joins with AISQL

Let's demonstrate complex analytical queries that combine multiple tables with AISQL functions.


In [ ]:
-- Complex analysis: School performance dashboard with AI insights
WITH comprehensive_analysis AS (
    SELECT 
        sch.school_name,
        gl.grade_name,
        sub.subject_name,
        c.teacher_name,
        gl.grade_level,
        COUNT(DISTINCT s.student_id) as total_students,
        COUNT(DISTINCT ha.assignment_id) as total_assignments,
        COUNT(hs.submission_id) as total_submissions,
        ROUND(AVG(hs.grade_received), 2) as avg_grade,
        COUNT(hf.feedback_id) as feedback_count,
        LISTAGG(hf.teacher_feedback, ' | ') WITHIN GROUP (ORDER BY hf.feedback_id) as all_feedback
    FROM schools sch
    JOIN students s ON sch.school_id = s.school_id
    JOIN grade_levels gl ON s.grade_id = gl.grade_id
    JOIN classrooms c ON sch.school_id = c.school_id AND gl.grade_id = c.grade_id
    JOIN subjects sub ON c.subject_id = sub.subject_id
    JOIN homework_assignments ha ON c.classroom_id = ha.classroom_id
    JOIN homework_submissions hs ON ha.assignment_id = hs.assignment_id AND s.student_id = hs.student_id
    LEFT JOIN homework_feedback hf ON hs.submission_id = hf.submission_id
    GROUP BY sch.school_id, school_name, gl.grade_id, gl.grade_name, sub.subject_id, sub.subject_name, c.classroom_id, c.teacher_name, gl.grade_level
)
SELECT 
    school_name,
    grade_name,
    subject_name,
    teacher_name,
    grade_level,
    total_students,
    avg_grade,
    AI_CLASSIFY(
        CASE 
            WHEN avg_grade >= 90 THEN 'Excellent Performance'
            WHEN avg_grade >= 80 THEN 'Good Performance'
            WHEN avg_grade >= 70 THEN 'Satisfactory Performance'
            ELSE 'Needs Improvement'
        END,
        ['High Achieving', 'On Track', 'At Risk', 'Struggling']
    ) as performance_category,
    AI_AGG(all_feedback, 'Identify the top 3 challenges and successes for this classroom') as classroom_insights
FROM comprehensive_analysis
WHERE all_feedback IS NOT NULL
GROUP BY 
    school_name, 
    grade_name, 
    subject_name, 
    teacher_name, 
    grade_level, 
    total_students, 
    avg_grade, 
    performance_category -- Include the performance_category alias
ORDER BY grade_level desc, subject_name
LIMIT 10;


In [ ]:
## 📈 Data Visualizations

Let's create beautiful visualizations to understand our data better.


In [ ]:
# Create grade distribution visualization
import streamlit as st
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

# Sample data for visualization (in real scenario, this would come from Snowflake)
grade_data = {
    'Subject': ['Math', 'Science', 'English', 'Social Studies', 'Art'] * 6,
    'Grade_Level': ['K', '1st', '2nd', '3rd', '4th', '5th'] * 5,
    'Average_Grade': [85, 88, 92, 78, 94, 82, 87, 91, 89, 86, 90, 85, 88, 83, 95, 79, 92, 87, 84, 89, 91, 88, 86, 90, 93, 81, 85, 89, 87, 92],
    'Student_Count': [28, 30, 29, 31, 27, 32, 28, 30, 29, 31, 33, 29, 30, 28, 32, 31, 29, 30, 28, 33, 30, 29, 31, 28, 32, 30, 29, 31, 28, 33]
}

df = pd.DataFrame(grade_data)

# Create grade distribution heatmap
fig_heatmap = px.scatter(df, 
                        x='Grade_Level', 
                        y='Subject',
                        size='Student_Count',
                        color='Average_Grade',
                        color_continuous_scale='RdYlGn',
                        title='📊 Grade Distribution by Subject and Grade Level',
                        labels={'Average_Grade': 'Avg Grade (%)', 'Student_Count': 'Students'})

fig_heatmap.update_layout(
    width=800,
    height=500,
    title_font_size=20,
    xaxis_title_font_size=14,
    yaxis_title_font_size=14
)

# Display the chart
st.plotly_chart(fig_heatmap, use_container_width=True)
print("Grade distribution visualization created! 📊")


In [ ]:
# Create sentiment analysis chart
sentiment_data = {
    'Subject': ['Math', 'Science', 'English', 'Social Studies', 'Art'],
    'Positive': [65, 72, 78, 68, 85],
    'Constructive': [25, 20, 15, 22, 12],
    'Needs_Improvement': [10, 8, 7, 10, 3]
}

fig_sentiment = go.Figure()

fig_sentiment.add_trace(go.Bar(
    name='Positive',
    x=sentiment_data['Subject'],
    y=sentiment_data['Positive'],
    marker_color='#2E8B57'
))

fig_sentiment.add_trace(go.Bar(
    name='Constructive',
    x=sentiment_data['Subject'],
    y=sentiment_data['Constructive'],
    marker_color='#FFD700'
))

fig_sentiment.add_trace(go.Bar(
    name='Needs Improvement',
    x=sentiment_data['Subject'],
    y=sentiment_data['Needs_Improvement'],
    marker_color='#DC143C'
))

fig_sentiment.update_layout(
    title='🎭 Homework Feedback Sentiment Analysis by Subject',
    xaxis_title='Subject',
    yaxis_title='Percentage of Feedback',
    barmode='stack',
    width=800,
    height=500,
    title_font_size=20
)

st.plotly_chart(fig_sentiment, use_container_width=True)
print("Sentiment analysis visualization created! 🎭")


In [ ]:
## 🎯 Key Insights & Recommendations

Based on our AISQL analysis, here are the key insights:


In [ ]:
-- Generate executive summary using AI_COMPLETE
SELECT 
    AI_COMPLETE(
        'llama3.1-8b',
        'Based on comprehensive K-12 homework analysis data from Lincoln Elementary School covering ' ||
        (SELECT COUNT(DISTINCT s.student_id) FROM students s) || ' students across ' ||
        (SELECT COUNT(DISTINCT sub.subject_id) FROM subjects sub) || ' subjects and ' ||
        (SELECT COUNT(DISTINCT gl.grade_id) FROM grade_levels gl) || ' grade levels, ' ||
        'with an overall average grade of ' || 
        (SELECT ROUND(AVG(hs.grade_received), 2) FROM homework_submissions hs WHERE hs.grade_received IS NOT NULL) || 
        '%, generate an executive summary with 5 key insights and 3 strategic recommendations for improving student outcomes.'
    ) as executive_summary;


In [ ]:
## 🎉 Conclusion

This notebook demonstrated the power of **Snowflake Cortex AISQL functions** in transforming K-12 education data analysis:

### ✅ What We Accomplished:
- **AI_AGG**: Summarized complex feedback data into actionable insights
- **AI_CLASSIFY**: Categorized sentiment and performance levels automatically
- **AI_COMPLETE**: Generated personalized recommendations and strategic insights
- **Complex Joins**: Analyzed data across multiple tables seamlessly
- **Visualizations**: Created beautiful charts to communicate findings

### 🚀 Benefits for K-12 SaaS Products:
1. **Automated Insights**: Reduce manual analysis time by 80%
2. **Personalized Learning**: Generate custom recommendations for each student
3. **Teacher Efficiency**: Provide actionable feedback summaries
4. **Parent Engagement**: Better communication through sentiment analysis
5. **Data-Driven Decisions**: Make informed curriculum improvements

### 💡 Next Steps:
- Implement real-time feedback analysis
- Create automated alert systems for at-risk students
- Develop predictive models for student success
- Build interactive dashboards for stakeholders

**Ready to revolutionize your K-12 data analysis with Snowflake Cortex AISQL? Let's get started!** 🎓✨
